In [ ]:
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pip install 'aif360[LawSchoolGPA]'


In [ ]:
custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']
complementary_colors = ['#a9d0f5', '#f3ba8c', '#98e698', '#f5a9a9', '#d8b5d8', '#d2a6a6', '#f5a9f2', '#d8d8d8']

full_palette = custom_colors + complementary_colors
sns.set_palette(full_palette)

plt.rcParams['axes.prop_cycle'] = plt.cycler(color=custom_colors)
plt.rcParams['axes.titlesize'] = 30
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['legend.fontsize'] = 15

In [ ]:
df = pd.read_pickle('../data/lifesnaps/lifesnapsv2.pkl')
df['id'] = df['id'].astype(str)
df

In [ ]:
df.columns

In [ ]:
protected_attributes = ['gender', 'bmi', 'age']
privileged_classes = [[0, 1, 0]]

data = StandardDataset(df,  label_name='label_stress_sema', favorable_classes=[0], protected_attribute_names=protected_attributes, privileged_classes=privileged_classes, features_to_drop=['id'])
metric = BinaryLabelDatasetMetric(data, unprivileged_groups=[{'gender': 1, 'bmi': 0, 'age': 1}], privileged_groups=[{'gender': 0, 'bmi': 1, 'age': 0}])

print("Consistency:", metric.consistency()) # Measures how consistently similar individuals receive similar predictions, regardless of their group. Higher consistency indicates less potential discrimination. Individual fairness metric from [1] that measures how similar the labels are for similar instances. 
# [1] (1, 2) R. Zemel, Y. Wu, K. Swersky, T. Pitassi, and C. Dwork, “Learning Fair Representations,” International Conference on Machine Learning, 2013.

In [ ]:
data = StandardDataset(df, label_name='label_stress_sema', favorable_classes=[0],
                       protected_attribute_names=protected_attributes, 
                       privileged_classes=privileged_classes,
                       features_to_drop=['id', 'date'])

# DataFrame to store the results
bias_columns_metric = pd.DataFrame(columns=['protected_attribute', 'consistency'])

# Loop through each protected attribute to calculate consistency
for attr in data.protected_attribute_names:
    print("\nEvaluating Consistency for {}".format(attr))

    # Calculate consistency metric
    consistency_metric = BinaryLabelDatasetMetric(data).consistency()

    # Updating DataFrame
    bias_columns_metric.loc[len(bias_columns_metric.index)] = [attr, consistency_metric[0]]
    print("Fairness Evaluation - Consistency: {}".format(consistency_metric[0]))